UNO Card Detection Application - Template Matching + ORB Feature Matching

In [39]:
#importing all the required modules
import cv2, os, time
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [40]:
#the directory containing the images (dataset)
unocardspath = "unocardimages"

In [41]:
#template matching for the upload a card option and after the captured card match
def match_card(input_img):
    unocardspath = "unocardimages"
    gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
    best_score = -1
    best_label = "Unknown"
    
    for filename in os.listdir(unocardspath):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            label = os.path.splitext(filename)[0]
            template = cv2.imread(os.path.join(unocardspath, filename), cv2.IMREAD_GRAYSCALE)
            template = cv2.resize(template, (gray.shape[1], gray.shape[0]))
            res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
            _, max_val, _, _ = cv2.minMaxLoc(res)

            if max_val > best_score:
                best_score = max_val
                best_label = label

    return best_label, best_score

In [42]:
#GUI for the UNO card detection - contains all the user options
class UNOCardGUI:
    def __init__(self, root):
        
        #initializing the whole window
        self.root = root
        self.root.title("UNO Card Detection")
        self.root.state("zoomed")
        self.root.configure(bg="#720137")

        #layout setup
        self.main_frame = tk.Frame(root, bg="#720137")
        self.main_frame.pack(fill="both", expand=True, padx=20, pady=20)

        self.left_frame = tk.Frame(self.main_frame, bg="#720137")
        self.left_frame.pack(side="left", fill="y", padx=20)

        self.right_frame = tk.Frame(self.main_frame, bg="#720137")
        self.right_frame.pack(side="right", fill="both", expand=True)

        
        #labels and buttons setup
        
        #title and subtitle
        self.title_label = tk.Label(self.left_frame, text="UNO Card Detection", font=("Forte", 24, "bold"), fg="white", bg="#720137")
        self.title_label.pack(pady=20)

        self.label = tk.Label(self.left_frame, text="Upload a card to have it be detected:", font=("Aptos", 14), fg="white", bg="#720137")
        self.label.pack(pady=10)
        
        #standard layout for all the buttons
        btn_style = {"font": ("Arial", 14), "fg": "black", "bg": "#F5D5E1", "width": 30, "height": 2, "bd": 0}

        #upload a single card
        self.upload_button = tk.Button(self.left_frame, text="Upload Card Image", command=self.upload_image, **btn_style) 
        self.upload_button.pack(pady=10)

        #upload mulitple cards
        self.multi_button = tk.Button(self.left_frame, text="Upload Multiple Cards", command=self.upload_multiple_images, **btn_style)
        self.multi_button.pack(pady=10)

        #stream from webcam and take pictures (screenshots)
        self.webcam_button = tk.Button(self.left_frame, text="Use Webcam - Capture", command=lambda: self.use_camera(0), **btn_style)
        self.webcam_button.pack(pady=10)

        #stream only - live from webcam 
        self.nocap_button = tk.Button(self.left_frame, text="Use Webcam - Live", command=lambda: self.use_camera_no_capture(0), **btn_style)
        self.nocap_button.pack(pady=10)

        #stream from laptop camera and take pictures (screenshots)
        self.laptopcam_button = tk.Button(self.left_frame, text="Use Laptop Camera - Capture", command=lambda: self.use_camera(1), **btn_style)
        self.laptopcam_button.pack(pady=10) 

        #canvas to display the uploaded cards results
        self.canvas = tk.Canvas(self.right_frame, width=300, height=500, bg="#720137", highlightbackground="white", highlightthickness=2)
        self.canvas.pack(pady=20)

        self.result_label = tk.Label(self.left_frame, text="", font=("Arial", 16, "bold"), fg="white", bg="#720137")
        self.result_label.pack(pady=10)

        #clear out the cards
        self.clear_button = tk.Button(self.right_frame, text="Clear", command=self.clear_canvas, font=("Arial", 12, "bold"),
                                      bg="#FF6B6B", fg="white", bd=0)
        self.clear_button.pack(pady=(40, 20))
        self.clear_button.pack_forget()

        self.image_list = []
        self.result_list = []
        self.current_index = 0

        #navigation of front and back when multiple cards are uploaded
        self.nav_frame = tk.Frame(self.right_frame, bg="#720137")
        nav_style = {"font": ("Arial", 10, "bold"), "bg": "#4CAF50", "fg": "white", "bd": 0}
        self.prev_button = tk.Button(self.nav_frame, text="←", command=self.show_prev_image, **nav_style)
        self.next_button = tk.Button(self.nav_frame, text="→", command=self.show_next_image, **nav_style)
        self.prev_button.pack(side="left", padx=10)
        self.next_button.pack(side="right", padx=10)
        self.nav_frame.pack_forget()

    #single image is uploaded
    def upload_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            img = cv2.imread(file_path)
            label, score = match_card(img)
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb).resize((300, 500))
            self.canvas.delete("all")
            self.canvas.image = ImageTk.PhotoImage(img_pil)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.canvas.image)
            self.result_label.config(text=f"Detected Card: {label} (Score: {score:%})")
            self.clear_button.pack()

    #multiple images are uploaded
    def upload_multiple_images(self):
        paths = filedialog.askopenfilenames()
        if paths:
            self.image_list = []
            self.result_list = []
            self.current_index = 0
            for path in paths:
                img = cv2.imread(path)
                label, score = match_card(img)
                self.image_list.append(path)
                self.result_list.append(f"{os.path.basename(path)} → {label} (Score: {score:%})")
            self.show_image_by_index(0)
            self.nav_frame.pack()
            self.clear_button.pack()

    
    def show_image_by_index(self, index):
        img = cv2.imread(self.image_list[index])
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img_rgb).resize((300, 500))
        self.canvas.delete("all")
        self.canvas.image = ImageTk.PhotoImage(img_pil)
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.canvas.image)
        self.result_label.config(text=self.result_list[index])

    def show_next_image(self):
        if self.current_index + 1 < len(self.image_list):
            self.current_index += 1
            self.show_image_by_index(self.current_index)

    def show_prev_image(self):
        if self.current_index - 1 >= 0:
            self.current_index -= 1
            self.show_image_by_index(self.current_index)

    #reset the layout
    def clear_canvas(self):
        self.canvas.delete("all")
        self.result_label.config(text="")
        self.clear_button.pack_forget()
        self.nav_frame.pack_forget()
        self.image_list.clear()
        self.result_list.clear()
        self.current_index = 0

    #webcam to detect + capture pictures
    def use_camera(self, cam_index):
        orb = cv2.ORB_create(nfeatures=2000)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)

        if not os.path.exists("captured"):
            os.makedirs("captured")

        #load all the images (templates)
        templates = {}
        for file in os.listdir(unocardspath):
            if file.endswith(".jpg") or file.endswith(".png"):
                label = os.path.splitext(file)[0]
                img = cv2.imread(os.path.join(unocardspath, file), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (200, 300))
                kp, des = orb.detectAndCompute(img, None)
                templates[label] = {"des": des}

        #template match the cards
        def match(des1):
            best_label = "Unknown"
            best_match = 0
            for lbl, data in templates.items():
                des2 = data["des"]
                if des1 is None or des2 is None:
                    continue
                matches = bf.knnMatch(des1, des2, k=2)
                good = [m for m, n in matches if m.distance < 0.75 * n.distance]
                if len(good) > best_match:
                    best_match = len(good)
                    best_label = lbl
            return best_label, best_match

        cap = cv2.VideoCapture(cam_index)  #0 for internal webcam and 1 for laptop camera
        if not cap.isOpened():
            print("Camera is not accessible.")
            return

        last_label = None
        flash_timer = 0

        #live frame processing loop
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            annotated = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(blur, 30, 150)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            found = False

            for cnt in sorted(contours, key=cv2.contourArea, reverse=True):
                if cv2.contourArea(cnt) < 5000:
                    continue
                approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
                if len(approx) != 4:
                    continue
                x, y, w, h = cv2.boundingRect(approx)
                roi = frame[y:y + h, x:x + w]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                roi_resized = cv2.resize(roi_gray, (200, 300))
                kp, des = orb.detectAndCompute(roi_resized, None)
                label, score = match(des)
                if score > 15:
                    found = True
                    cv2.rectangle(annotated, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(annotated, f"{label} ({score})", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    if label != last_label:
                        ts = int(time.time())
                        filename = f"captured/{label}_score{score}.jpg"
                        cv2.imwrite(filename, annotated)
                        print("[image saved]", filename)
                        last_label = label
                        flash_timer = 3
                    break

            if not found:
                last_label = None

            if flash_timer > 0:
                overlay = annotated.copy()
                white = np.full_like(overlay, 255)
                annotated = cv2.addWeighted(white, 0.6, overlay, 0.4, 0)
                flash_timer -= 1

            cv2.imshow("UNO Card Detector", annotated)
            if cv2.waitKey(1) & 0xFF == 27:
                break
    
    def use_camera_no_capture(self, cam_index):
        orb = cv2.ORB_create(nfeatures=2000)
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    
        templates = {}
        for file in os.listdir(unocardspath):
            if file.endswith(".jpg") or file.endswith(".png"):
                label = os.path.splitext(file)[0]
                img = cv2.imread(os.path.join(unocardspath, file), cv2.IMREAD_GRAYSCALE)
                img = cv2.resize(img, (200, 300))
                kp, des = orb.detectAndCompute(img, None)
                templates[label] = {"des": des}
    
        def match(des1):
            scores = {}
            for lbl, data in templates.items():
                des2 = data["des"]
                if des1 is None or des2 is None:
                    continue
                matches = bf.knnMatch(des1, des2, k=2)
                good = [m for m, n in matches if m.distance < 0.75 * n.distance]
                scores[lbl] = len(good)
            if not scores:
                return "Unknown", 0
            best_label = max(scores, key=scores.get)
            return best_label, scores[best_label]
    
        cap = cv2.VideoCapture(cam_index)
        if not cap.isOpened():
            print("Camera is not accessible.")
            return
    
        while True:
            ret, frame = cap.read()
            if not ret:
                break
    
            annotated = frame.copy()
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray, (5, 5), 0)
            edges = cv2.Canny(blur, 50, 150)
            contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            adaptive_thresh = cv2.adaptiveThreshold(
                blur, 255,
                cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                cv2.THRESH_BINARY_INV,
                11, 2
            )

            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
            morph = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)
    
            for cnt in sorted(contours, key=cv2.contourArea, reverse=True):
                if cv2.contourArea(cnt) < 5000:
                    continue
    
                approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
                if len(approx) < 4:
                    continue
    
                x, y, w, h = cv2.boundingRect(cnt)
                aspect_ratio = float(w) / h
                if aspect_ratio < 0.5 or aspect_ratio > 0.75:
                    continue  
                roi = frame[y:y + h, x:x + w]
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                roi_gray = cv2.equalizeHist(roi_gray)
                roi_resized = cv2.resize(roi_gray, (200, 300))
    
                kp, des = orb.detectAndCompute(roi_resized, None)
                label, score = match(des)
    
                if score > 10:
                    cv2.rectangle(annotated, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    cv2.putText(annotated, f"{label} ({score})", (x, y - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    break
    
            cv2.imshow("UNO Card Detector (Live Only)", annotated)
            if cv2.waitKey(1) & 0xFF == 27:
                break
    
        cap.release()
        cv2.destroyAllWindows()


In [43]:
#launching the GUI application
if __name__ == "__main__":
    root = tk.Tk()
    app = UNOCardGUI(root)
    root.mainloop()